Import some of the important libraries to create a neural network using pytorch library

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
import random

!wget -q https://raw.githubusercontent.com/jhaluska10/kaggle/main/kaggle.json
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download  keystroke-dynamics-challenge-1
! unzip  keystroke-dynamics-challenge-1.zip


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  0% 0.00/53.7k [00:00<?, ?B/s]
100% 53.7k/53.7k [00:00<00:00, 61.0MB/s]
Archive:  keystroke-dynamics-challenge-1.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               



Begin to parse the data for the neural network, creating press-press times, release-press times, release-release times, and hold durations. Also create the whole latency for each word ("united" and "states"). Finally, standardizes the inputs of the neural net.

In [2]:
keystroke_info = pd.read_csv('train.csv')
for i in range(13):
  if(i != 0):
    keystroke_info['PP-'+str(i)] = keystroke_info['press-'+str(i)] - keystroke_info['press-'+str(i-1)]
    keystroke_info['RP-'+str(i)] = keystroke_info['release-'+str(i)] - keystroke_info['press-'+str(i-1)]
    keystroke_info['RR-'+str(i)] = keystroke_info['release-'+str(i)] - keystroke_info['release-'+str(i-1)]
  keystroke_info['HD-'+str(i)] = keystroke_info['release-'+str(i)] - keystroke_info['press-'+str(i)]

#MAY WANT TO ADD DIGRAPHS laTER. AM UNSURE AS OF NOW'
Latency_metric = ['HD', 'PP' , 'RR' , 'RP']
#Latency_metric = [ 'PP' , 'RR' , 'RP']

for val in Latency_metric:
        lists = [col for col in keystroke_info.columns if val in str(col)]
        #print(lists)
        keystroke_info[str(val)+'-mean'] = keystroke_info[lists].mean(axis=1)
        keystroke_info[str(val)+'-std'] = keystroke_info[lists].std(axis=1)

for i in range(1,13):
    keystroke_info['DG-'+str(i)] = keystroke_info['release-'+str(i)] - keystroke_info['press-'+str(i-1)]

for i in range(2,13):
    keystroke_info['TG-'+str(i)] = keystroke_info['release-'+str(i)] - keystroke_info['press-'+str(i-2)]
keystroke_info["united_latency"] =   keystroke_info['release-5'] - keystroke_info['press-0']
keystroke_info["states_latency"] =   keystroke_info['release-12'] - keystroke_info['press-7']
keystroke_info
results = keystroke_info["user"]

results_tensor = [[] for _ in results]
for i in range(110):
  for j in range(len(results_tensor)):
    if j // 8 == i:
      results_tensor[j].append(1)
    else:
      results_tensor[j].append(0)
results_tensor = torch.Tensor(results_tensor)
keystroke_info = keystroke_info.iloc[:,1:]
inputs = nn.functional.normalize(torch.tensor(keystroke_info.values).float())

#bins
#normalize, batching, after optimizer.step see the weights, is it changing at all

Create the activation function as well as the structure of the neural net that attempts to categorize keystroke entries for each possible user. This neural net uses the Adam optimizer and a leaky ReLU activation function. The drawbacks of this method are that it could be difficult to add a new user into this neural net structure.

In [7]:
active = nn.LeakyReLU(0.1)

class Net(nn.Module):
    def __init__(self,nhidden=109,nin=108):
        super(Net, self).__init__()
        self.nhidden = nhidden
        self.nin = nin
        hidden_nodes = []
        for h in range(nhidden):
            hidden_nodes.append(torch.nn.Parameter((torch.rand(self.nin+1,1)-.5)*2/torch.sqrt(torch.tensor(2))))
        self.hidden_nodes = nn.ParameterList(hidden_nodes)
        self.output = nn.Linear(nhidden, 110, bias=True)

    def forward(self, x):
        x = torch.cat((x,-torch.ones((x.shape[0],1))),axis=1)
        ys = []
        for h in self.hidden_nodes:
            ys.append(torch.matmul(x,h))
        y = torch.cat(ys,dim=1)
        y = active(y) # end of the hidden layer forward step?
        y = torch.softmax(self.output(y),dim=1)
        return y

    def predict(self,x):
        return torch.round(self.forward(x))

This code creates the instances of the above defined neural nets and tests their accuracy which is output at the bottom of the code cell.

In [18]:
import copy
net = Net()
criterion = nn.CrossEntropyLoss()
nepochs = 1000
optimizer = optim.Adam(net.parameters(), lr=0.01)
train_loss = []
val_loss = []
models = []
labels = results_tensor.float()

# print(labels.shape)

for epoch in range(nepochs):
    optimizer.zero_grad()
    outputs = net(inputs)
    # print(outputs.shape)
    loss = criterion(outputs.float(), labels)
    loss.backward()
    optimizer.step()

    train_loss.append(float(loss.data.numpy()))
    val_loss.append(float(criterion(net(inputs).float(),labels).data.numpy()))
    #models.append(copy.deepcopy(net))

    if epoch % 100 == 0:
      print(f'[{epoch}] loss:',loss.data.numpy())


outputs = net(inputs)
correct = 0
for i in range(len(outputs)):
  if i // 8 == torch.argmax(outputs[i]).item():
    correct += 1
print("Accuracy")
print(correct / len(outputs))
  #print(str(i) + ": " + str(torch.argmax(outputs[i]).item()))

[0] loss: 4.700463
[100] loss: 4.4645467
[200] loss: 4.4311085
[300] loss: 4.4207644
[400] loss: 4.4196205
[500] loss: 4.419212
[600] loss: 4.418983
[700] loss: 4.418839
[800] loss: 4.413723
[900] loss: 4.4104943
Accuracy
0.30454545454545456


We implement binning of the data here and see if the resulting model performs better.

In [9]:
bins = 10
for i in range(len(inputs)):
  inputs[i] = torch.round(inputs[i]/(1/bins)) * (1/bins)
inputs

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.1000, 0.1000, 0.2000],
        [0.0000, 0.0000, 0.0000,  ..., 0.1000, 0.1000, 0.1000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2000, 0.1000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.1000, 0.2000, 0.1000],
        [0.0000, 0.0000, 0.0000,  ..., 0.1000, 0.2000, 0.1000],
        [0.0000, 0.0000, 0.0000,  ..., 0.1000, 0.2000, 0.1000]])

In [20]:
import copy
net = Net()
criterion = nn.CrossEntropyLoss()
nepochs = 1000
optimizer = optim.Adam(net.parameters(), lr=0.01)
train_loss = []
val_loss = []
models = []
labels = results_tensor.float()

# print(labels.shape)

for epoch in range(nepochs):
    optimizer.zero_grad()
    outputs = net(inputs)
    # print(outputs.shape)
    loss = criterion(outputs.float(), labels)
    loss.backward()
    optimizer.step()

    train_loss.append(float(loss.data.numpy()))
    val_loss.append(float(criterion(net(inputs).float(),labels).data.numpy()))
    #models.append(copy.deepcopy(net))

    if epoch % 100 == 0:
      print(f'[{epoch}] loss:',loss.data.numpy())


outputs = net(inputs)
correct = 0
for i in range(len(outputs)):
  if i // 8 == torch.argmax(outputs[i]).item():
    correct += 1
print("Accuracy")
print(correct / len(outputs))
  #print(str(i) + ": " + str(torch.argmax(outputs[i]).item()))

[0] loss: 4.700495
[100] loss: 4.450742
[200] loss: 4.415797
[300] loss: 4.4051437
[400] loss: 4.397783
[500] loss: 4.3944445
[600] loss: 4.3941236
[700] loss: 4.385017
[800] loss: 4.379192
[900] loss: 4.379036
Accuracy
0.3352272727272727
